In [1]:
%matplotlib inline

import os
import sys
import pickle
import datacube
import numpy as np
import pandas as pd
from pandas import DataFrame
import geopandas as gpd
import geopandas.tools
from shapely.geometry import Point
import xarray as xr
import ipyleaflet as ipyl
from ipyleaflet import Map, Marker
import ipywidgets as ipyw
import json
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display
from datacube.utils import masking
from datacube.utils import geometry 
from datacube.utils.geometry import CRS
from datacube.helpers import write_geotiff

## Scrape gauge data from the BOM website
This is where you use the python script located in the Scripts folder. There are a few versions of this python script, some only get the gauge name and location, some actually generate a map of the gauges and you can click one and it retrieves the data of that gauge. That script is called dea_bom.py which is in my Flood-Mapping repository and it was written in September 2019 by Kirill at GA. 

In [2]:
#Kirill at GA wrote me this script in 2019 that retrieves data from the Water Data Online website
#dea_bom_storage retrieves the name and location of gauges that measure Storage Level
sys.path.append('Scripts')
import station_finder
stations = station_finder.get_stations()

# Preview the first five stations loaded
print(f'{len(stations)} stations loaded; e.g.:')
stations[:5]

613 stations loaded; e.g.:


[namespace(name='10 Mile Brook Dam Water Level Daily Value, South W',
           pos=(-33.963617, 115.1248985)),
 namespace(name='284', pos=(-32.74362461, 151.9612514)),
 namespace(name='2km N Warringee Pt', pos=(-35.67660246, 139.2502685)),
 namespace(name='3km W Pt McLeay', pos=(-35.51481282, 139.0647519)),
 namespace(name='40A', pos=(-32.79766496, 151.743779))]

## Extract position and name of the gauges 
the variable called stations is a list of namespace objects (SimpleNamespace). Inside each namespace object is the position and name of the gauge. We can get this information out with a loop:

In [3]:
#make some empty lists to append the names and locations into
pos_list = []
name_list = []
#Write a loop to extract each 'pos' tuple and 'name' string from the namespaces in the list
for i in range(len(stations)):
    pos = stations[i].pos
    name = stations[i].name
    pos_list.append(pos)
    name_list.append(name)
    
#Make a pandas dataframe out of these 2 lists (pos_list and name_list)
df = DataFrame(name_list,columns=['staion_name']) #create a dataframe with one column
df['location'] = pos_list #make another column

#split up the 'location' tuples into 2 columns, one for lat, one for lon
df[['lat', 'lon']] = pd.DataFrame(df['location'].tolist(), index=df.index)

#Make the data spatial by making a GeoDataFrame. Geopandas has a function points_from_xy to do this. 
st_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat)) 
#make sure the lat and lon are the right way around in the output. lon should be first (eg: 115.124 -33.963)

st_gdf

,staion_name,location,lat,lon,geometry
0,"10 Mile Brook Dam Water Level Daily Value, Sou...","(-33.963617, 115.1248985)",-33.963617,115.124899,POINT (115.12490 -33.96362)
1,284,"(-32.74362461, 151.9612514)",-32.743625,151.961251,POINT (151.96125 -32.74362)
2,2km N Warringee Pt,"(-35.67660246, 139.2502685)",-35.676602,139.250269,POINT (139.25027 -35.67660)
3,3km W Pt McLeay,"(-35.51481282, 139.0647519)",-35.514813,139.064752,POINT (139.06475 -35.51481)
4,40A,"(-32.79766496, 151.743779)",-32.797665,151.743779,POINT (151.74378 -32.79766)
...,...,...,...,...,...
608,YARRADDA LAGOON,"(-34.5846999999996, 145.8236)",-34.584700,145.823600,POINT (145.82360 -34.58470)
609,Yabba Ck at Borumba Dam HW,"(-26.5096751869416, 152.58186478)",-26.509675,152.581865,POINT (152.58186 -26.50968)
610,Yarrawonga Weir,"(-36.0103075, 146.091535)",-36.010308,146.091535,POINT (146.09153 -36.01031)
611,Yealering,"(-32.59697157, 117.62690338)",-32.596972,117.626903,POINT (117.62690 -32.59697)


## Open the named reservoirs dataset
See the notebook called 'Making the Reservoirs dataset' to see how I got to this point. Basically, it's made from the national surface water layer that you can download for free on the Digital Earth Australia website. And then in ArcGIS I extracted the polygons that were labelled 'Reservoir', made a shapefile out of it and then put it on this sandbox account, deleted a few columns, deleted any reservoirs that didn't have a name attached to them and obtained the centroid of the polygons to get the lat and lon columns. 

In [4]:
gdf = gpd.read_file('Reservoirs/Named_Reservoirs/Named_Reservoirs.shp')

#zip the lat and lon up into a tuple. We need this later to match the gauge to its reservoir
gdf['location'] = list(zip(gdf.lat, gdf.lon)) 
gdf = gdf[['NAME', 'SHAPE_Area', 'Area_calc', 'lat', 'lon', 'location', 'geometry']]
gdf

,NAME,SHAPE_Area,Area_calc,lat,lon,location,geometry
0,WILSON RESERVOIR,0.000028,270561.736,-37.510018,144.018549,"(-37.51001775248238, 144.01854920219148)","POLYGON ((144.01821 -37.51455, 144.01719 -37.5..."
1,SUGARLOAF RESERVOIR,0.000438,4284887.270,-37.672015,145.305467,"(-37.672015168559675, 145.30546667448777)","POLYGON ((145.31084 -37.68275, 145.31059 -37.6..."
2,GREEN LAKE,0.000715,7111145.660,-36.435338,144.834187,"(-36.435338183618754, 144.83418677261471)","POLYGON ((144.82094 -36.43662, 144.82081 -36.4..."
3,LAKE TAYLOR,0.000490,4853573.620,-36.783437,142.383614,"(-36.783436899772425, 142.38361438566585)","POLYGON ((142.39295 -36.77087, 142.39255 -36.7..."
4,UPPER STONY CREEK RESERVOIR,0.000051,500287.353,-37.818353,144.203635,"(-37.81835304591356, 144.20363450185346)","POLYGON ((144.21140 -37.81367, 144.21152 -37.8..."
...,...,...,...,...,...,...,...
668,KIRAR WEIR,0.000240,2681688.020,-25.299279,151.063109,"(-25.299278958136348, 151.0631085211492)","POLYGON ((151.07836 -25.34552, 151.07834 -25.3..."
669,MARIAN WEIR,0.000134,1544632.180,-21.146641,148.885758,"(-21.14664099340445, 148.88575832646802)","POLYGON ((148.82987 -21.17723, 148.82926 -21.1..."
670,MARY BARRAGE,0.000115,1283147.610,-25.627152,152.613932,"(-25.62715200544585, 152.61393206577483)","POLYGON ((152.59687 -25.65457, 152.59709 -25.6..."
671,TINANA BARRAGE,0.000012,129235.966,-25.571684,152.713346,"(-25.571684097336387, 152.71334611851472)","POLYGON ((152.71974 -25.56929, 152.71973 -25.5..."


## See if the gauges are on the reservoirs
This is a visual check to see how many of the gauges match up to the reservoirs, ie are in the same place. I have only taken gauges that were labelled 'storage level' by BOM, but there is another class of gauges called 'depth' or something which if we included those, might get the rest of the reservoirs that don't have gauges in the 'water storage' category but do have 'depth' gauges. 

In [ ]:
#Converting the data to json
data = json.loads(gdf.to_json())
st_data = json.loads(st_gdf.to_json())

map = ipyl.Map(center=[-28, 148], zoom=7)

label = ipyw.Label(layout=ipyw.Layout(width='100%'))

#This part draws the polygons for the reservoirs
for feature in data['features']:
    feature['properties']['style'] = {
        'color': 'grey',
        'weight': 1,
        'fillColor': 'grey',
        'fillOpacity': 0.5
    }
layer = ipyl.GeoJSON(data=data, hover_style={'fillColor': 'red'})

def click_handler(event=None, feature=None, id=None, properties=None):
    label.value = str(properties['NAME'])
    
    
layer.on_hover(click_handler)
map.add_layer(layer) #add the polygons to the map

#This part is the markers for the gauges
features = st_data['features']
#I got this off the internet for adding each gauge location
for i in range(len(features)):
    location=(features[i]['geometry']['coordinates'][1],features[i]['geometry']['coordinates'][0])

    marker = Marker(location=location)

    map.add_layer(marker)
    
map


ipyw.VBox([map, label])

## Merge the gauge data with the reservoir data by location
I want to add the gauge name and gauge location for each reservoir. We will probably do this with a spatial join. I don't know how to code that so I'm just going to save these 2 files out and do it in ArcGIS.  

## Save the files out so you can do a spatial join in ArcGIS

In [ ]:
polys = gdf.drop(columns = ['location'])
points = st_gdf.drop(columns = ['location'])

#you want the points to have the same crs as the reservoir (GDA94). 
#I don't actually know if the points data is GDA94 when it comes straight off the BoM website, but let's just assume it is. 
points.crs = polys.crs

output = 'polys'
polys.to_file(output)

output = 'points'
points.to_file(output)